# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

In [3]:
# Get data
data = spark.read.csv('hack_data.csv',
                      header=True,inferSchema=True)

In [4]:
data.printSchema() # Location is useless. They use VPNs

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [5]:
data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [6]:
# We want to group attacks, knowing there are 2 or 3 hackers
# No label. This is unsupervised learning!

In [7]:
# Prepare data for ML
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [9]:
# We will pass all columns through the assembler!
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
                                       'Bytes Transferred',
                                       'Kali_Trace_Used',
                                       'Servers_Corrupted',
                                       'Pages_Corrupted',
                                       'WPM_Typing_Speed'],
                            outputCol='features')

In [10]:
finalData = assembler.transform(data)

In [11]:
finalData.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
# Scale your Features! Make sure every single one
# is on the same order of magnitude!
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol='features',
                        outputCol='scaledFeatures')

In [14]:
# Fit (collect stddev info) and transform accordingly
scalerModel = scaler.fit(finalData)
finalData = scalerModel.transform(finalData)

In [15]:
finalData.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), scaledFeatures=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963]))]

In [17]:
# Initiate the model with 2 and 3 clusters
kmeans_2 = KMeans(featuresCol='scaledFeatures',k=2)
kmeans_3 = KMeans(featuresCol='scaledFeatures',k=3)

In [18]:
# Fit the models
kModel_2 = kmeans_2.fit(finalData)
kModel_3 = kmeans_3.fit(finalData)

In [20]:
# Evaluate your model -> "Within Set Sum of Squared Errors"
wssse_2 = kModel_2.summary.trainingCost
print(wssse_2)
wssse_3 = kModel_3.summary.trainingCost
print(wssse_3)

# Value is not helpful because the data got scaled!

601.7707512676691
434.75507308487596


In [22]:
# View centroid coordinates
centers_2 = kModel_2.clusterCenters()
print('2 CENTERS')
print(centers_2)
centers_3 = kModel_3.clusterCenters()
print('3 CENTERS')
print(centers_3)

2 CENTERS
[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
       3.28474   ]), array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
       5.26676612])]
3 CENTERS
[array([3.05623261, 2.95754486, 1.99757683, 3.2079628 , 4.49941976,
       3.26738378]), array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
       5.26676612]), array([2.93719177, 2.88492202, 0.        , 3.19938371, 4.52857793,
       3.30407351])]


In [23]:
# Check number of attacks (rows) with 2 hackers
results_2 = kModel_2.transform(finalData)
results_2.select('prediction').show()

+----------+
|prediction|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 20 rows



In [26]:
print(results_2.filter("Prediction = 0").count())
print(results_2.filter("Prediction = 1").count())
print(results_2.filter("Prediction = 2").count())

# OR
# results_2.groupBy('Prediction').count().show()

167
167
0


In [27]:
# Check number of attacks (rows) with 3 hackers
results_3 = kModel_3.transform(finalData)
results_3.select('prediction').show()

+----------+
|prediction|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 20 rows



In [29]:
print(results_3.filter("Prediction = 0").count())
print(results_3.filter("Prediction = 1").count())
print(results_3.filter("Prediction = 2").count())

# OR
# results_3.groupBy('Prediction').count().show()

88
167
79


In [ ]:
# There were only 2 hackers!